In [1]:
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from collections import defaultdict
from IPython.display import display
from fuzzywuzzy import fuzz, process
from ITUtils import country_conflicts_finder  # Assuming you have this util function
import pandarallel
# show all columns
pd.set_option('display.max_rows', None)

In [2]:
# pick a country and import the conflicts
adm = 'D'
file = 'expanded_combined_tables_conflicts_lettersatnames.csv'
filepath = os.path.join('.', 'adm_conflicts', adm, file)
df = pd.read_csv(filepath)
display(df.head())
print(df.columns)

/tmp/ipykernel_613200/1914993252.py:5: DtypeWarning: Columns (20,22,32,33,34,35,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


,com_el.ntc_id,com_el.tgt_ntc_id,com_el.adm,com_el.ntwk_org,com_el.sat_name,com_el.long_nom,com_el.prov,com_el.d_rcv,com_el.st_cur,orbit.orb_id,...,TPA1.2055.475-2055.725_R,TPA1.401.89525-401.90475_R,TPA1.2237.0-2238.0_E,TPA1.2065.575-2065.825_R,TPA1.401.9501-401.9699_E,TPA1.401.8901-401.9099_R,TPA1.401.95525-401.96475_E,TPA1.401.8901-401.9099_E,TPA1.401.89525-401.90475_E,TPA1.2202.4-2203.4_E
0,119545250,NaN,D,NaN,LS-4,NaN,9.1/IA,02.02.2021,50,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,119545250,NaN,D,NaN,LS-4,NaN,9.1/IA,02.02.2021,50,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,119545250,NaN,D,NaN,LS-4,NaN,9.1/IA,02.02.2021,50,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,119545250,NaN,D,NaN,LS-4,NaN,9.1/IA,02.02.2021,50,5.0,...,NaN,2%RE,NaN,NaN,4%EE,4%RE,2%EE,4%EE,2%EE,NaN
4,119545250,NaN,D,NaN,LS-4,NaN,9.1/IA,02.02.2021,50,2.0,...,NaN,2%RE,NaN,NaN,4%EE,4%RE,2%EE,4%EE,2%EE,NaN


Index(['com_el.ntc_id', ' com_el.tgt_ntc_id', ' com_el.adm',
       ' com_el.ntwk_org', ' com_el.sat_name', ' com_el.long_nom',
       ' com_el.prov', ' com_el.d_rcv', ' com_el.st_cur', ' orbit.orb_id',
       ' orbit.nbr_sat_pl', ' orbit.apog_km', ' orbit.perig_km',
       ' orbit.op_ht_km', ' s_beam.emi_rcp', ' s_beam.beam_name',
       ' grp.grp_id', ' grp.freq_min', ' grp.freq_max', ' grp.bdwdth',
       ' grp.d_inuse', ' grp.d_reg_limit', ' grp.d_prot_eff', ' grp.f_biu',
       ' emiss.seq_no', ' emiss.pwr_ds_max', ' emiss.design_emi',
       ' carrier_fr.freq_carr', ' channel.bandwidth', ' channel.freq_min',
       ' channel.freq_max', 'tpaconflicts', 'percentoverlap',
       'TPA1.401.95525-401.96475_R', 'TPA1.401.9501-401.9699_R',
       'TPA1.2055.475-2055.725_R', 'TPA1.401.89525-401.90475_R',
       'TPA1.2237.0-2238.0_E', 'TPA1.2065.575-2065.825_R',
       'TPA1.401.9501-401.9699_E', 'TPA1.401.8901-401.9099_R',
       'TPA1.401.95525-401.96475_E', 'TPA1.401.8901-401.9099_E',

In [3]:
# funcition to extract the table
def summary_table(df):
    # columns to keep
    cols = [
        ' com_el.sat_name',
        ' s_beam.beam_name',
        ' carrier_fr.freq_carr',
        ' channel.bandwidth',
        'tpaconflicts',
        'percentoverlap'
    ]
    lookupnames = [
        'UHFUP fc=401.96 bw=9K5',
        'UHFUP fc=401.96 bw=19K8',
        'SUP fc=2055.6 bw=250K',
        'UHFUP fc=401.90 bw=9K5',
        'SDN fc=2237.5 bw=1M',
        'SUP fc=2065.7 bw=250k',
        'UHFDN fc=401.96 bw=19K8',
        'UHFUP fc=401.90 bw=19K8',
        'UHFDN fc=401.96 bw=9K5',
        'UHFDN fc=401.90 bw=19K8',
        'UHFDN fc=401.90 bw=9K5',
        'SDN fc=2202.9 bw=1M'
    ]
    # get the columns
    df1 = df[cols]
    # add a literal column 
    df1['Overlaps'] = None
    def functiontoapply(row):
        confidx = row['tpaconflicts'].split(':')
        confidx = confidx[:,-1]
        perc = row['percentoverlap'].split(':')
        overlap = []
        for i in range(len(perc)):
            over = str(lookupnames[confidx[i]]) +' '+ str(perc[i])+'% overlap'
            overlap.append(over)
        row['Overlaps'] = ' : '.join(overlap)
        return row
    # apply in parallel to all rows
    if len(df1>24):
        df1 = df1.parallel_apply(functiontoapply, axis=1)
    else:
        df1 = df1.apply(functiontoapply, axis=1)
    
    # drop columns 
    df1 = df1.drop(['tpaconflicts', 'percentoverlap'], axis=1)
    # rename columns
    df1 = df1.rename(columns={
        ' com_el.sat_name': 'Sat Name',
        ' s_beam.beam_name': 'Beam Name',
        ' carrier_fr.freq_carr' : 'Carrier Freq. (fc)',
        ' channel.bandwidth' : 'Bandwidth',}) # this is calculated from the emission designator code e.g. 19k8XXX
    return df1

            

In [4]:
# apply to dataframe
table = summary_table(df)
# init pandaralle
# Initialize pandarallel
num_logical_processors = os.cpu_count()
pandarallel.initialize(nb_workers=num_logical_processors, progress_bar=True)
# export 
outpath = os.path.join('.', 'adm_conflicts', adm,'conflicts_summary.csv')
table.to_csv(outpath, index=False)

/tmp/ipykernel_613200/3489561791.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Overlaps'] = None


TypeError: '>' not supported between instances of 'str' and 'int'